In [1]:
from halide import *
from scipy.misc import imread, imsave

In [2]:
x, y, c = Var("x"), Var("y"), Var("c")
image_path = "./rgb.png"

In [9]:
# Now we'll express a multi-stage pipeline that blurs an image
# first horizontally, and then vertically.
input = Buffer(imread(image_path))
assert input.type() == UInt(8)

input_16 = Func("input_16")
input_16[x,y,c] = cast(UInt(16),input[x,y,c])

blur_x = Func("blur_x")
blur_x[x,y,c] = (input_16[x-1,y,c] + 2 * input_16[x,y,c] + input_16[x+1,y,c]) /4

blur_y = Func("blur_y")
blur_y[x,y,c] = (blur_x[x,y-1,c] + 2 * blur_x[x,y,c] + blur_x[x,y+1,c])

output = Func("output")
output[x,y,c] = cast(UInt(8),blur_x[x,y,c])

result = Buffer(UInt(8), input.width() - 2, input.height() - 2, 3)
result.set_min(1,1)
output.realize(result)

In [10]:
result_data = buffer_to_ndarray(result)
print("result.shape", result_data.shape)

imsave("blurry_parrot_1.png", result_data)
print("Created blurry_parrot_1.pnt")

result.shape (1278, 766, 3)
Created blurry_parrot_1.pnt


In [11]:
input = Buffer(imread(image_path))
assert input.type() == UInt(8)

clamped = Func("clamped")
clamped_x = clamp(x, 0, input.width() - 1)
clamped_y = clamp(y, 0, input.height() - 1)
clamped[x,y,c] = input[clamped_x, clamped_y, c]

input_16 = Func("input_16")
input_16[x,y,c] = cast(UInt(16), clamped[x,y,c])

blur_x = Func("blur_x")
blur_x[x,y,c] = (input_16[x-1,y,c] + 2 * input_16[x,y,c] + input_16[x+1,y,c]) / 4

blur_y = Func("blur_y")
blur_y[x,y,c] = (blur_x[x,y-1,c] + 2 * blur_x[x,y,c] + blur_x[x,y+1,c])/4

output = Func("output")
output[x,y,c] = cast(UInt(8), blur_y[x,y,c])

result = output.realize(input.width(), input.height(), 3)
result_data = buffer_to_ndarray(result)
print("result.shape", result_data.shape)

imsave("blurry_parrot_2.png", result_data)
print("Created blurry_parrot_2.png")

result.shape (1280, 768, 3)
Created blurry_parrot_2.png


In [12]:
print("Success!")

Success!
